In [1]:
import sys
sys.path.append("..")

In [2]:
import os

In [3]:
import torch
import torch.optim as optim

In [4]:
import train_semantic_segmentation as semseg

In [5]:
DATA_PATHS = {
    'source': os.path.join('../../data/VOCdevkit/VOC2012/JPEGImages'),
    'segmentation': os.path.join('../../data/VOCdevkit/VOC2012/SegmentationClass'),
    'train': os.path.join('../../data/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt'),
    'val': os.path.join('../../data/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt')
}

In [6]:
train_loader, val_loader, test_loader = semseg.load_data(
    DATA_PATHS['source'],
    DATA_PATHS['segmentation'],
    DATA_PATHS['train'],
    DATA_PATHS['val'],
    DATA_PATHS['val']
)

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
learning_rate = 0.007
epochs = 50

In [9]:
model = semseg.DeepLabModel(input_channels=3, num_classes=21).to(device)

conv1.weight -> low_level_net.conv1.weight
bn1.weight -> low_level_net.bn1.weight
bn1.bias -> low_level_net.bn1.bias
bn1.running_mean -> low_level_net.bn1.running_mean
bn1.running_var -> low_level_net.bn1.running_var
layer1.0.conv1.weight -> low_level_net.low_level_block.net.0.net.conv1.weight
layer1.0.bn1.weight -> low_level_net.low_level_block.net.0.net.bn1.weight
layer1.0.bn1.bias -> low_level_net.low_level_block.net.0.net.bn1.bias
layer1.0.bn1.running_mean -> low_level_net.low_level_block.net.0.net.bn1.running_mean
layer1.0.bn1.running_var -> low_level_net.low_level_block.net.0.net.bn1.running_var
layer1.0.conv2.weight -> low_level_net.low_level_block.net.0.net.conv2.weight
layer1.0.bn2.weight -> low_level_net.low_level_block.net.0.net.bn2.weight
layer1.0.bn2.bias -> low_level_net.low_level_block.net.0.net.bn2.bias
layer1.0.bn2.running_mean -> low_level_net.low_level_block.net.0.net.bn2.running_mean
layer1.0.bn2.running_var -> low_level_net.low_level_block.net.0.net.bn2.running_var

In [10]:
criterion = semseg.segmentation_cross_entropy_loss(size_average=None,
                                                   ignore_index=255,
                                                   device=device)
optimizer = optim.SGD(semseg.differential_learning_rates(model, [
                          ((model.feature_detection_layers, ), 1),
                          ((model.spatial_pyramid_pooling, model.decoder), 10)
                      ], learning_rate),
                      momentum=0.9,
                      weight_decay=5e-4,
                      nesterov=False)
scheduler = semseg.PolynomialLearningRateScheduler(optimizer,
                                                   learning_rate,
                                                   epochs,
                                                   len(train_loader))

In [11]:
def log_statistics_to_notebook(stats):
    print(stats)

In [ ]:
semseg.training_loop(model,
                     train_loader,
                     val_loader,
                     criterion,
                     optimizer,
                     scheduler,
                     device,
                     epochs=epochs,
                     statistics_callback=log_statistics_to_notebook,
                     save_callback=None,
                     start_epoch=0,
                     best_accumulated_miou=0)